In [2]:
import psycopg2
import sys
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import time , datetime , date

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as off


## Funciones de utilidad

In [ ]:
def postgre_connect(host,db):
    conn_string = "host='" + host + "' dbname='"+db+"' user='postgres' password='postgres'"
    print ("Connecting to database\n	->%s" % (conn_string))
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print ("Connected!\n")
    return (conn , cursor)

In [6]:
def analyze(prod_name, df  , features ):

    data = []

    ff = go.Scatter(
        x=df.index,
        y=df[features],
        name = prod_name,
        line = dict(color = '#17BECF'),
        opacity = 0.8)
    data.append(ff)
        
    layout = dict(
        title=prod_name,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(),
            type='date'
        )
    )

    fig = dict(data=data, layout=layout)
    off.plot(fig , filename= prod_name + '.html')


In [17]:
def get_frame(table):
    df = pd.read_sql("select * from " + table ,conn)
    print ('Leidos:',len(df))
    return df

def clean_data(frame):
    df2 = frame.copy()
    df2['datetime'] = df2.apply(lambda x : datetime.combine( x['event_date'] , x['event_time'] ) , axis = 1)
    df2['event_type'] = df2['event_type'].apply(lambda x : x.strip())
    df2['event_time'] = df2['event_time'].apply(lambda x : time.strftime(x , '%H:%M:%S.%f'))
    df2['event_date'] = df2['event_date'].apply(lambda x : date.strftime(x , '%Y-%m-%d'))
    df2 = df2.set_index(['datetime'])
    return df2


In [29]:
pickle_storage = './dataframes/'
pickle_extension = '.pkl'

def get_pickle(dbname,name):
    filename = pickle_storage + dbname + '_'+ name + pickle_extension
    return pd.read_pickle(filename)

def put_pickle(dbname,name, frame):
    filename = pickle_storage  + dbname + '_'+ name + pickle_extension
    frame.to_pickle(filename)

In [32]:

def _get_unique_datetimes_for_set_second_window(frame , window_seconds):
    framex = frame.copy()

    def get_window(x , size):
        r = x / size
        return r * size
    
    def get_trunked_datetime(d , t):
        dd = datetime.strptime(d+t,'%Y-%m-%d%H:%M:%S.%f')
        return datetime(dd.year , dd.month, dd.day , dd.hour, dd.minute, get_window(dd.second,window_seconds))

    applied = framex.apply(lambda x : get_trunked_datetime(x.event_date ,  x.event_time) , axis=1) 

    return np.unique(applied)

def get_unique_datetimes_by_5_seconds(frame):
    return _get_unique_datetimes_for_set_second_window(frame,5)

def get_unique_datetimes_by_1_minute(frame):
    framex = frame.copy()

    def get_trunked_datetime(d , t):
        dd = datetime.strptime(d+t,'%Y-%m-%d%H:%M:%S.%f')
        return datetime(dd.year , dd.month, dd.day , dd.hour, dd.minute, 0)


    applied = framex.apply(lambda x : get_trunked_datetime(x.event_date ,  x.event_time) , axis=1) 

    return np.unique(applied)

def get_unique_datetimes_by_1_second(frame):
    framex = frame.copy()

    def get_trunked_datetime(d , t):
        dd = datetime.strptime(d+t,'%Y-%m-%d%H:%M:%S.%f')
        return datetime(dd.year , dd.month, dd.day , dd.hour, dd.minute, dd.second)


    applied = framex.apply(lambda x : get_trunked_datetime(x.event_date ,  x.event_time) , axis=1) 

    return np.unique(applied)

In [34]:
def generate_final_set(frame, func_datetimes ):
    
    initial_ask_prices = []
    initial_bid_prices = []
    final_ask_prices = []
    final_bid_prices = []
  
    
    starts = []
    ends = []

    datetimes = func_datetimes(frame)
    
    for i in range(0 , len(datetimes)):

        if i < len(datetimes) -1:

            start = datetimes[i]
            end = datetimes[i+1]

            starts.append(start)
            ends.append(end)
            
            mini_set = frame[ (frame.index >= start) & (frame.index < end) ]

            ## Agrego el price en este momento
            ##
            initial_ask_price = mini_set['ask_price'].head(1).values[0]
            initial_bid_price = mini_set['bid_price'].head(1).values[0]
            last_ask_price = mini_set['ask_price'].tail(1).values[0]
            last_bid_price = mini_set['bid_price'].tail(1).values[0]
            
            initial_ask_prices.append(initial_ask_price)
            initial_bid_prices.append(initial_bid_price)
            final_ask_prices.append(last_ask_price)
            final_bid_prices.append(last_bid_price)
            
          
           
    obj = {'start':starts , 'end': ends,           
           'initial_ask_price':initial_ask_prices ,
           'initial_bid_price':initial_bid_prices ,
           'final_ask_price':final_ask_prices ,
           'final_bid_price':final_bid_prices }
             
    n_columns=['start' , 'end',
        'initial_ask_price','initial_bid_price','final_ask_price','final_bid_price'] 
    
    return pd.DataFrame(obj , columns=n_columns) 

## Cargando los datos de la base (esto se ejecuta una sola vez por la latencia de la red)

Los libros que nos interesa son:
<li>bitfinex_btcusd</li>
<li>bitfinex_xrpbtc</li>
<li>bitfinex_xrpusd</li>

#### Abrimos

In [40]:
(conn , cursor ) = postgre_connect('nitsuga.com.ar','gtrader')

Connecting to database
	->host='nitsuga.com.ar' dbname='gtrader' user='postgres' password='postgres'
Connected!



#### Leemos

In [41]:
df_raw_bitfinex_btcusd = get_frame('bitfinex_btcusd')
df_raw_bitfinex_xrpbtc = get_frame('bitfinex_xrpbtc')
df_raw_bitfinex_xrpusd = get_frame('bitfinex_xrpusd')

Leidos: 1874322
Leidos: 1936674
Leidos: 1899275


#### Cerramos

In [42]:
cursor.close()
conn.close()

## Limpiando la data

In [43]:
df_clean_bitfinex_btcusd = clean_data(df_raw_bitfinex_btcusd)
df_clean_bitfinex_xrpbtc = clean_data(df_raw_bitfinex_xrpbtc)
df_clean_bitfinex_xrpusd = clean_data(df_raw_bitfinex_xrpusd)

## Guardamos en pickle los datos

In [44]:
put_pickle('raw','bitfinex_btcusd', df_raw_bitfinex_btcusd)
put_pickle('clean','bitfinex_btcusd', df_clean_bitfinex_btcusd)
put_pickle('raw','bitfinex_xrpbtc', df_raw_bitfinex_xrpbtc)
put_pickle('clean','bitfinex_xrpbtc', df_clean_bitfinex_xrpbtc)
put_pickle('raw','bitfinex_xrpusd', df_raw_bitfinex_xrpusd)
put_pickle('clean','bitfinex_xrpusd', df_clean_bitfinex_xrpusd)

## NO EJECUTAR - Recuperamos los frames de Pickle

In [31]:
df2 = get_pickle('cleaned','bitfinex_ethbtc')

## NO EJECUTAR - Separamos los tipos de eventos

## ----------------------------------------------------------------------------------------

In [19]:
trades = df2[(df2['event_type'] == 'TRADE') ]
bids = df2[(df2['event_type'] == 'BID') ]
asks = df2[(df2['event_type'] == 'ASK') ]

In [22]:
## trades =0 , gracias bruno..
print( len(trades))
print( len(bids))
print( len(asks))

0
986278
1115096


## ----------------------------------------------------------------------------------------

In [45]:
df_disc_1_sec_bitfinex_btcusd = generate_final_set(df_clean_bitfinex_btcusd, get_unique_datetimes_by_1_second )
df_disc_1_sec_bitfinex_xrpbtc = generate_final_set(df_clean_bitfinex_xrpbtc, get_unique_datetimes_by_1_second )
df_disc_1_sec_bitfinex_xrpusd = generate_final_set(df_clean_bitfinex_xrpusd, get_unique_datetimes_by_1_second )

In [46]:
put_pickle('discretized_1_second','bitfinex_btcusd', df_disc_1_sec_bitfinex_btcusd)
put_pickle('discretized_1_second','bitfinex_xrpbtc', df_disc_1_sec_bitfinex_xrpbtc)
put_pickle('discretized_1_second','bitfinex_xrpusd', df_disc_1_sec_bitfinex_xrpusd)

## Visualizamos los datos

In [13]:
#analyze('bitfinex_ethbtc' ,trades, 'event_price')

,entry_id,event_date,event_time,event_type,event_price,event_size,ask_price,ask_size,bid_price,bid_size
datetime,,,,,,,,,,
2018-01-12 20:57:29.837070,1,2018-01-12,20:57:29.837070,BID,0.091336,1,0.091338,1,0.091336,1
2018-01-12 20:57:29.845138,2,2018-01-12,20:57:29.845138,ASK,0.091338,1,0.091338,1,0.091336,1
2018-01-12 20:57:30.579718,3,2018-01-12,20:57:30.579718,BID,0.091336,1,0.091338,1,0.091336,1
2018-01-12 20:57:30.586886,4,2018-01-12,20:57:30.586886,ASK,0.091338,1,0.091338,1,0.091336,1
2018-01-12 20:57:30.622927,5,2018-01-12,20:57:30.622927,ASK,0.091337,1,0.091337,1,0.091336,1
